In [1]:
import pandas as pd

In [57]:
df = pd.read_csv('/Users/siddhantgond/Desktop/6THSEM/Project_Elective/Adaptive-Network-Intrusion-Detection-System/Dataset/Normal_dataset.csv')

/var/folders/p0/fzpcz2t93td2j063ysbwphbr0000gn/T/ipykernel_10664/2206728934.py:1: DtypeWarning: Columns (5,30,39) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/Users/siddhantgond/Desktop/6THSEM/Project_Elective/Adaptive-Network-Intrusion-Detection-System/Dataset/Normal_dataset.csv')


In [8]:
df.head().to_latex('output.tex')

In [58]:
# First split the frame.time column into temporary columns
df['year'] = df['frame.time'].str.strip().str.split(n=1, expand=True)[0]
df['time'] = df['frame.time'].str.strip().str.split(n=1, expand=True)[1]

# Now rename the columns to desired names
df = df.rename(columns={'year': 'frame_year', 'time': 'frame_date'})

# Apply conversion only for valid year entries (optional step)
df['frame_year'] = df['frame_year'].apply(lambda x: int(x) if x.isdigit() and len(x) == 4 else x)

# Display the result to verify
print(df[['frame.time', 'frame_year', 'frame_date']].head())

                  frame.time frame_year          frame_date
0   2022 00:51:59.554403000        2022  00:51:59.554403000
1   2022 05:54:06.889467000        2022  05:54:06.889467000
2   2021 19:15:21.234421000        2021  19:15:21.234421000
3   2021 04:58:55.411604000        2021  04:58:55.411604000
4   2021 05:20:28.623968000        2021  05:20:28.623968000


In [59]:
df.drop('frame.time', axis=1, inplace=True)

In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 64 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   ip.src_host                150000 non-null  object 
 1   ip.dst_host                150000 non-null  object 
 2   arp.dst.proto_ipv4         150000 non-null  object 
 3   arp.opcode                 150000 non-null  float64
 4   arp.hw.size                150000 non-null  object 
 5   arp.src.proto_ipv4         150000 non-null  object 
 6   icmp.checksum              150000 non-null  float64
 7   icmp.seq_le                150000 non-null  float64
 8   icmp.transmit_timestamp    150000 non-null  float64
 9   icmp.unused                150000 non-null  float64
 10  http.file_data             150000 non-null  float64
 11  http.content_length        150000 non-null  float64
 12  http.request.uri.query     150000 non-null  float64
 13  http.request.method        15

In [23]:
df.drop('Frame_year', axis=1, inplace=True)
df.drop('Frame_date', axis=1, inplace=True)

In [60]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 64 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   ip.src_host                150000 non-null  object 
 1   ip.dst_host                150000 non-null  object 
 2   arp.dst.proto_ipv4         150000 non-null  object 
 3   arp.opcode                 150000 non-null  float64
 4   arp.hw.size                150000 non-null  object 
 5   arp.src.proto_ipv4         150000 non-null  object 
 6   icmp.checksum              150000 non-null  float64
 7   icmp.seq_le                150000 non-null  float64
 8   icmp.transmit_timestamp    150000 non-null  float64
 9   icmp.unused                150000 non-null  float64
 10  http.file_data             150000 non-null  float64
 11  http.content_length        150000 non-null  float64
 12  http.request.uri.query     150000 non-null  float64
 13  http.request.method        15

# Dataset feature selection

In [63]:
import pandas as pd
import numpy as np
import joblib
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

# Step 1: Load Dataset (Assuming `td.csv` Contains Only Normal Samples)
df_normal = pd.read_csv("td.csv")

# Step 2: Drop Unnecessary Columns (Labels, High-Cardinality, Non-Informative)
columns_to_drop = [
    'ip.src_host', 'ip.dst_host',            # IP addresses (Not generalizable)
    'arp.dst.proto_ipv4', 'arp.src.proto_ipv4',  # IP addresses
    'tcp.payload', 'dns.qry.name', 'mqtt.msg', 'mqtt.topic',  # High-cardinality text features
    'frame_date', 'frame_year',              # Timestamps (Redundant)
    'tcp.checksum', 'tcp.ack_raw', 'tcp.options'  # Redundant/Verification fields
]
df_normal.drop(columns=columns_to_drop, axis=1, inplace=True, errors='ignore')

# Step 3: Convert Object Columns to Numeric Format
df_normal['tcp.len'] = pd.to_numeric(df_normal['tcp.len'], errors='coerce')
df_normal['dns.qry.name.len'] = pd.to_numeric(df_normal['dns.qry.name.len'], errors='coerce')

# Step 4: Convert `tcp.dstport` to Meaningful Categories (Only If Present)
if 'tcp.dstport' in df_normal.columns:
    df_normal['tcp.dstport'] = pd.to_numeric(df_normal['tcp.dstport'], errors='coerce')

    def categorize_port(port):
        if pd.isna(port):
            return np.nan
        elif port <= 1023:
            return 0  # Well-known ports
        elif port <= 49151:
            return 1  # Registered ports
        else:
            return 2  # Dynamic ports

    df_normal['tcp.dstport_category'] = df_normal['tcp.dstport'].apply(categorize_port)
    df_normal.drop('tcp.dstport', axis=1, inplace=True)

# Step 5: Apply Label Encoding for Categorical Columns (Only Low-Cardinality)
categorical_cols = df_normal.select_dtypes(include=['object']).columns
label_encoders = {}

for col in categorical_cols:
    if df_normal[col].nunique() < 10:  # Safe for Label Encoding
        le = LabelEncoder()
        df_normal[col] = le.fit_transform(df_normal[col].astype(str))
        label_encoders[col] = le  # Store encoder for consistency
    else:
        df_normal.drop(columns=[col], inplace=True)  # Drop high-cardinality categorical features

# Step 6: Handle Missing Values (Instead of Dropping Rows)
df_normal.fillna(df_normal.median(numeric_only=True), inplace=True)

# Step 7: Remove Highly Correlated Columns (Correlation > 0.95)
corr_matrix = df_normal.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
df_normal.drop(columns=to_drop, axis=1, inplace=True)

# Step 8: Scale the Data Using MinMaxScaler (Recommended for Autoencoder)
scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df_normal)
df_scaled = pd.DataFrame(df_scaled, columns=df_normal.columns)

# Step 9: Save Preprocessed Dataset & Artifacts
df_scaled.to_csv('processed_training_dataset.csv', index=False)
joblib.dump(scaler, "scaler.pkl")
joblib.dump(label_encoders, "label_encoders.pkl")

# Print Summary
print(f"Final number of features: {df_scaled.shape[1]}")
print("Preprocessing complete. Processed dataset saved as 'processed_training_dataset.csv'.")
print("Scaler saved as 'scaler.pkl' and Label Encoders saved as 'label_encoders.pkl'.")

Final number of features: 45
Preprocessing complete. Processed dataset saved as 'processed_training_dataset.csv'.
Scaler saved as 'scaler.pkl' and Label Encoders saved as 'label_encoders.pkl'.


In [64]:
df_scaled.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 45 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   arp.opcode               150000 non-null  float64
 1   icmp.checksum            150000 non-null  float64
 2   icmp.seq_le              150000 non-null  float64
 3   icmp.transmit_timestamp  150000 non-null  float64
 4   icmp.unused              150000 non-null  float64
 5   http.file_data           150000 non-null  float64
 6   http.content_length      150000 non-null  float64
 7   http.request.uri.query   150000 non-null  float64
 8   http.request.method      150000 non-null  float64
 9   http.referer             150000 non-null  float64
 10  http.request.full_uri    150000 non-null  float64
 11  http.request.version     150000 non-null  float64
 12  http.response            150000 non-null  float64
 13  http.tls_port            150000 non-null  float64
 14  tcp.

Feature Importance 

In [65]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import mutual_info_classif
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import os

# Create output directory for plots if it doesn't exist
os.makedirs('plots', exist_ok=True)

# Load the dataset
print("Loading dataset...")
df = pd.read_csv('processed_training_dataset.csv')

# Check if 'Attack_label' exists in the dataset
# If not, we'll need to simulate it for demonstration purposes
if 'Attack_label' not in df.columns:
    print("Warning: 'Attack_label' not found in dataset. Creating a simulated label for demonstration.")
    # This is just a placeholder - in a real scenario, you would use the actual label
    # For demonstration, we'll assume 5% of data points are attacks
    np.random.seed(42)
    df['Attack_label'] = np.random.choice([0, 1], size=len(df), p=[0.95, 0.05])

# Get features and target
X = df.drop('Attack_label', axis=1) if 'Attack_label' in df.columns else df
y = df['Attack_label'] if 'Attack_label' in df.columns else None

feature_names = X.columns.tolist()

# 1. Calculate Mutual Information scores
print("Calculating Mutual Information scores...")
if y is not None:
    mi_scores = mutual_info_classif(X, y, random_state=42)
    mi_scores = pd.Series(mi_scores, index=feature_names)
    mi_scores = mi_scores.sort_values(ascending=False)
    
    # Plot Mutual Information scores
    plt.figure(figsize=(12, 10))
    mi_scores.plot(kind='bar')
    plt.title('Feature Importance Based on Mutual Information')
    plt.xlabel('Features')
    plt.ylabel('Mutual Information Score')
    plt.tight_layout()
    plt.savefig('plots/mutual_information_scores.png', dpi=300)
    plt.close()
    
    # Identify important features based on MI scores
    # Keep features that have MI scores above the mean or a threshold
    threshold = mi_scores.mean()
    important_features_mi = mi_scores[mi_scores > threshold].index.tolist()
    print(f"Selected {len(important_features_mi)} features based on MI scores (threshold: {threshold:.6f})")
else:
    print("Skipping Mutual Information calculation as 'Attack_label' is not available")
    mi_scores = pd.Series([0] * len(feature_names), index=feature_names)  # Placeholder
    important_features_mi = feature_names  # Keep all features

# 2. Perform PCA to determine variance contribution
print("Performing PCA analysis...")
# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Apply PCA
pca = PCA()
pca.fit(X_scaled)

# Calculate cumulative explained variance
cumulative_variance = np.cumsum(pca.explained_variance_ratio_)

# Plot the scree plot (explained variance)
plt.figure(figsize=(12, 6))
plt.plot(range(1, len(pca.explained_variance_ratio_) + 1), 
         pca.explained_variance_ratio_, 'o-', linewidth=2, color='blue')
plt.title('Scree Plot')
plt.xlabel('Principal Component')
plt.ylabel('Variance Explained')
plt.savefig('plots/pca_scree_plot.png', dpi=300)
plt.close()

# Plot cumulative explained variance
plt.figure(figsize=(12, 6))
plt.plot(range(1, len(cumulative_variance) + 1), 
         cumulative_variance, 'o-', linewidth=2, color='blue')
plt.axhline(y=0.95, color='r', linestyle='--', label='95% Explained Variance')
plt.title('Cumulative Explained Variance')
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')
plt.legend()
plt.grid(True)
plt.savefig('plots/pca_cumulative_variance.png', dpi=300)
plt.close()

# Determine number of components needed for 95% variance
n_components_95 = np.argmax(cumulative_variance >= 0.95) + 1
print(f"Number of components needed for 95% variance: {n_components_95}")

# Get feature importance from PCA (using the first component as simplification)
pca_importance = np.abs(pca.components_[0])
pca_importance = pd.Series(pca_importance, index=feature_names)
pca_importance = pca_importance.sort_values(ascending=False)

# Plot PCA-based feature importance
plt.figure(figsize=(12, 10))
pca_importance.plot(kind='bar')
plt.title('Feature Importance Based on First PCA Component')
plt.xlabel('Features')
plt.ylabel('PCA Loading (absolute value)')
plt.tight_layout()
plt.savefig('plots/pca_feature_importance.png', dpi=300)
plt.close()

# 3. Combine feature selection methods
# If we have MI scores, use both methods; otherwise, just use PCA
if y is not None:
    # Scale importance scores to 0-1 range for both methods
    mi_scores_scaled = (mi_scores - mi_scores.min()) / (mi_scores.max() - mi_scores.min())
    pca_importance_scaled = (pca_importance - pca_importance.min()) / (pca_importance.max() - pca_importance.min())
    
    # Calculate combined score (equal weight to MI and PCA)
    combined_scores = pd.DataFrame({
        'MI_Score': mi_scores,
        'MI_Score_Scaled': mi_scores_scaled,
        'PCA_Loading': pca_importance,
        'PCA_Loading_Scaled': pca_importance_scaled,
    })
    
    combined_scores['Combined_Score'] = combined_scores['MI_Score_Scaled'] * 0.7 + combined_scores['PCA_Loading_Scaled'] * 0.3
    combined_scores = combined_scores.sort_values('Combined_Score', ascending=False)
    
    # Select features above threshold (e.g., mean of combined scores)
    threshold = combined_scores['Combined_Score'].mean()
    selected_features = combined_scores[combined_scores['Combined_Score'] > threshold].index.tolist()
else:
    # If no MI scores, just use PCA-based importance
    combined_scores = pd.DataFrame({
        'PCA_Loading': pca_importance,
    })
    combined_scores = combined_scores.sort_values('PCA_Loading', ascending=False)
    
    # Select top features based on PCA loading
    threshold = pca_importance.mean()
    selected_features = pca_importance[pca_importance > threshold].index.tolist()

print(f"Selected {len(selected_features)} features based on combined scores")
print("Top 10 selected features:")
for i, feature in enumerate(selected_features[:10]):
    print(f"{i+1}. {feature}")

# 4. Generate LaTeX file with results
print("Generating LaTeX report...")
with open('feature_importance.tex', 'w') as f:
    f.write("\\documentclass{article}\n")
    f.write("\\usepackage{graphicx}\n")
    f.write("\\usepackage{booktabs}\n")
    f.write("\\usepackage{longtable}\n")
    f.write("\\usepackage{geometry}\n")
    f.write("\\geometry{margin=1in}\n")
    f.write("\\title{Feature Importance Analysis for Anomaly Detection}\n")
    f.write("\\author{GORK AI System}\n")
    f.write("\\date{\\today}\n")
    f.write("\\begin{document}\n")
    f.write("\\maketitle\n")
    
    f.write("\\section{Feature Importance Analysis}\n")
    
    # 1. Introduction
    f.write("\\subsection{Introduction}\n")
    f.write("This report presents the feature importance analysis for the dataset used in the Layer 1 (Autoencoder-VAE) of the GORK AI system. ")
    f.write("We analyzed a total of " + str(len(feature_names)) + " features to determine their significance in anomaly detection.\n")
    
    # 2. Mutual Information Analysis
    f.write("\\subsection{Mutual Information Analysis}\n")
    if y is not None:
        f.write("Mutual Information (MI) measures the dependency between features and the target variable (Attack\\_label). ")
        f.write("Higher MI scores indicate more informative features for detecting anomalies.\n")
        
        f.write("\\begin{figure}[ht]\n")
        f.write("\\centering\n")
        f.write("\\includegraphics[width=\\textwidth]{plots/mutual_information_scores.png}\n")
        f.write("\\caption{Feature Importance Based on Mutual Information Scores}\n")
        f.write("\\label{fig:mi_scores}\n")
        f.write("\\end{figure}\n")
        
        # MI scores table
        f.write("\\begin{longtable}{lrr}\n")
        f.write("\\toprule\n")
        f.write("Feature & MI Score & Normalized Score \\\\\n")
        f.write("\\midrule\n")
        
        for feature, score in mi_scores.items():
            normalized_score = (score - mi_scores.min()) / (mi_scores.max() - mi_scores.min())
            f.write(f"{feature} & {score:.6f} & {normalized_score:.6f} \\\\\n")
        
        f.write("\\bottomrule\n")
        f.write("\\caption{Mutual Information Scores for All Features}\n")
        f.write("\\end{longtable}\n")
    else:
        f.write("Mutual Information analysis was skipped as the 'Attack\\_label' was not available in the dataset.\n")
    
    # 3. PCA Analysis
    f.write("\\subsection{Principal Component Analysis}\n")
    f.write("Principal Component Analysis (PCA) was used to understand the variance contribution of each feature and dimensionality reduction potential.\n")
    
    f.write("\\begin{figure}[ht]\n")
    f.write("\\centering\n")
    f.write("\\includegraphics[width=0.8\\textwidth]{plots/pca_scree_plot.png}\n")
    f.write("\\caption{Scree Plot Showing Variance Explained by Each Principal Component}\n")
    f.write("\\label{fig:pca_scree}\n")
    f.write("\\end{figure}\n")
    
    f.write("\\begin{figure}[ht]\n")
    f.write("\\centering\n")
    f.write("\\includegraphics[width=0.8\\textwidth]{plots/pca_cumulative_variance.png}\n")
    f.write("\\caption{Cumulative Explained Variance by Principal Components}\n")
    f.write("\\label{fig:pca_cumulative}\n")
    f.write("\\end{figure}\n")
    
    f.write(f"Based on the PCA analysis, {n_components_95} components are needed to explain 95\\% of the variance in the data.\n")
    
    f.write("\\begin{figure}[ht]\n")
    f.write("\\centering\n")
    f.write("\\includegraphics[width=\\textwidth]{plots/pca_feature_importance.png}\n")
    f.write("\\caption{Feature Importance Based on First PCA Component}\n")
    f.write("\\label{fig:pca_importance}\n")
    f.write("\\end{figure}\n")
    
    # PCA importance table
    f.write("\\begin{longtable}{lr}\n")
    f.write("\\toprule\n")
    f.write("Feature & PCA Loading (abs) \\\\\n")
    f.write("\\midrule\n")
    
    for feature, importance in pca_importance.items():
        f.write(f"{feature} & {importance:.6f} \\\\\n")
    
    f.write("\\bottomrule\n")
    f.write("\\caption{PCA-based Feature Importance Scores}\n")
    f.write("\\end{longtable}\n")
    
    # 4. Combined Feature Selection
    f.write("\\subsection{Combined Feature Selection}\n")
    
    if y is not None:
        f.write("We combined the results from Mutual Information and PCA to obtain a more robust feature selection. ")
        f.write("Features were selected based on a combined score that gives 70\\% weight to MI and 30\\% weight to PCA importance.\n")
        
        # Combined scores table
        f.write("\\begin{longtable}{lrrr}\n")
        f.write("\\toprule\n")
        f.write("Feature & MI Score & PCA Loading & Combined Score \\\\\n")
        f.write("\\midrule\n")
        
        for feature in combined_scores.index[:30]:  # Show top 30 features
            row = combined_scores.loc[feature]
            f.write(f"{feature} & {row['MI_Score']:.6f} & {row['PCA_Loading']:.6f} & {row['Combined_Score']:.6f} \\\\\n")
        
        f.write("\\bottomrule\n")
        f.write("\\caption{Top 30 Features Based on Combined Importance Score}\n")
        f.write("\\end{longtable}\n")
    else:
        f.write("Since the 'Attack\\_label' was not available, feature selection was based solely on PCA loadings.\n")
        
        # PCA-based selection table
        f.write("\\begin{longtable}{lr}\n")
        f.write("\\toprule\n")
        f.write("Feature & PCA Loading \\\\\n")
        f.write("\\midrule\n")
        
        for feature in combined_scores.index[:30]:  # Show top 30 features
            importance = combined_scores.loc[feature, 'PCA_Loading']
            f.write(f"{feature} & {importance:.6f} \\\\\n")
        
        f.write("\\bottomrule\n")
        f.write("\\caption{Top 30 Features Based on PCA Importance}\n")
        f.write("\\end{longtable}\n")
    
    # 5. Conclusion
    f.write("\\subsection{Conclusion}\n")
    f.write(f"Based on our analysis, we identified {len(selected_features)} features as important for anomaly detection. ")
    f.write("These features will be used in the Layer 1 (Autoencoder-VAE) of the GORK AI system.\n")
    
    f.write("\\paragraph{Selected Features:}\n")
    f.write("\\begin{itemize}\n")
    for feature in selected_features[:min(20, len(selected_features))]:  # List top 20 features
        f.write(f"\\item {feature}\n")
    
    if len(selected_features) > 20:
        f.write(f"\\item ... {len(selected_features) - 20} more features\n")
    
    f.write("\\end{itemize}\n")
    
    f.write("\\end{document}\n")

print("Feature importance analysis complete. Results saved to 'feature_importance.tex'")

Loading dataset...
Calculating Mutual Information scores...
Selected 10 features based on MI scores (threshold: 0.000470)
Performing PCA analysis...
Number of components needed for 95% variance: 20
Selected 13 features based on combined scores
Top 10 selected features:
1. tcp.dstport_category
2. mbtcp.trans_id
3. tcp.ack
4. mqtt.ver
5. tcp.connection.synack
6. mbtcp.len
7. mqtt.conflags
8. mqtt.conack.flags
9. tcp.connection.rst
10. http.tls_port
Generating LaTeX report...
Feature importance analysis complete. Results saved to 'feature_importance.tex'


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import mutual_info_classif
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Load dataset
df = pd.read_csv("processed_training_dataset.csv")

# Check feature names
feature_names = df.columns.tolist()
print(f"Total features: {len(feature_names)}")

# Compute feature correlation heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(df.corr(), cmap='coolwarm', annot=False, fmt=".2f")
plt.title("Feature Correlation Heatmap")
plt.savefig("plots/feature_correlation_heatmap.png", dpi=300)
plt.close()

# Compute Mutual Information scores (if Attack_label is available)
if 'Attack_label' in df.columns:
    X = df.drop('Attack_label', axis=1)
    y = df['Attack_label']
    
    mi_scores = mutual_info_classif(X, y, random_state=42)
    mi_scores = pd.Series(mi_scores, index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)

    # Plot MI scores
    plt.figure(figsize=(12, 6))
    mi_scores.plot(kind='bar', color='blue')
    plt.title('Feature Importance Based on Mutual Information')
    plt.xlabel('Features')
    plt.ylabel('Mutual Information Score')
    plt.xticks(rotation=90)
    plt.tight_layout()
    plt.savefig("plots/mutual_information_distribution.png", dpi=300)
    plt.close()
else:
    print("Warning: 'Attack_label' not found, skipping MI computation.")

# PCA Analysis
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df)

pca = PCA()
pca.fit(df_scaled)

# Scree Plot (Explained Variance)
plt.figure(figsize=(12, 6))
plt.plot(range(1, len(pca.explained_variance_ratio_) + 1), 
         pca.explained_variance_ratio_, 'o-', linewidth=2, color='blue')
plt.title('Scree Plot')
plt.xlabel('Principal Component')
plt.ylabel('Variance Explained')
plt.grid()
plt.savefig("plots/pca_scree_plot.png", dpi=300)
plt.close()

# Cumulative Explained Variance Plot
cumulative_variance = np.cumsum(pca.explained_variance_ratio_)
plt.figure(figsize=(12, 6))
plt.plot(range(1, len(cumulative_variance) + 1), cumulative_variance, 'o-', linewidth=2, color='blue')
plt.axhline(y=0.95, color='r', linestyle='--', label='95% Explained Variance')
plt.title('Cumulative Explained Variance')
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Variance')
plt.legend()
plt.grid()
plt.savefig("plots/pca_cumulative_variance.png", dpi=300)
plt.close()

# PCA Feature Importance
pca_importance = np.abs(pca.components_[0])
pca_importance = pd.Series(pca_importance, index=feature_names)
pca_importance = pca_importance.sort_values(ascending=False)

plt.figure(figsize=(12, 6))
pca_importance.plot(kind='bar', color='red')
plt.title('Feature Importance Based on First PCA Component')
plt.xlabel('Features')
plt.ylabel('PCA Loading (absolute value)')
plt.xticks(rotation=90)
plt.tight_layout()
plt.savefig("plots/pca_feature_importance.png", dpi=300)
plt.close()

# Histogram for Feature Distributions
df.hist(figsize=(15, 12), bins=50, edgecolor="black")
plt.suptitle("Feature Distribution Histograms")
plt.savefig("plots/feature_histograms.png", dpi=300)
plt.close()

print("Feature importance visualization complete. Plots saved in 'plots/' directory.")


Total features: 45
Feature importance visualization complete. Plots saved in 'plots/' directory.


In [2]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import mutual_info_classif
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Ensure output directory exists after kernel restart
os.makedirs('plots', exist_ok=True)

# Reload dataset
df = pd.read_csv("processed_training_dataset.csv")

# Check feature names
feature_names = df.columns.tolist()
print(f"Total features: {len(feature_names)}")

# Compute feature correlation heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(df.corr(), cmap='coolwarm', annot=False, fmt=".2f")
plt.title("Feature Correlation Heatmap")
plt.savefig("plots/feature_correlation_heatmap.png", dpi=300)
plt.close()

# Compute Mutual Information scores (if Attack_label is available)
if 'Attack_label' in df.columns:
    X = df.drop('Attack_label', axis=1)
    y = df['Attack_label']
    
    mi_scores = mutual_info_classif(X, y, random_state=42)
    mi_scores = pd.Series(mi_scores, index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)

    # Plot MI scores
    plt.figure(figsize=(12, 6))
    mi_scores.plot(kind='bar', color='blue')
    plt.title('Feature Importance Based on Mutual Information')
    plt.xlabel('Features')
    plt.ylabel('Mutual Information Score')
    plt.xticks(rotation=90)
    plt.tight_layout()
    plt.savefig("plots/mutual_information_distribution.png", dpi=300)
    plt.close()
else:
    print("Warning: 'Attack_label' not found, skipping MI computation.")

# PCA Analysis
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df)

pca = PCA()
pca.fit(df_scaled)

# Scree Plot (Explained Variance)
plt.figure(figsize=(12, 6))
plt.plot(range(1, len(pca.explained_variance_ratio_) + 1), 
         pca.explained_variance_ratio_, 'o-', linewidth=2, color='blue')
plt.title('Scree Plot')
plt.xlabel('Principal Component')
plt.ylabel('Variance Explained')
plt.grid()
plt.savefig("plots/pca_scree_plot.png", dpi=300)
plt.close()

# Cumulative Explained Variance Plot
cumulative_variance = np.cumsum(pca.explained_variance_ratio_)
plt.figure(figsize=(12, 6))
plt.plot(range(1, len(cumulative_variance) + 1), cumulative_variance, 'o-', linewidth=2, color='blue')
plt.axhline(y=0.95, color='r', linestyle='--', label='95% Explained Variance')
plt.title('Cumulative Explained Variance')
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Variance')
plt.legend()
plt.grid()
plt.savefig("plots/pca_cumulative_variance.png", dpi=300)
plt.close()

# PCA Feature Importance
pca_importance = np.abs(pca.components_[0])
pca_importance = pd.Series(pca_importance, index=feature_names)
pca_importance = pca_importance.sort_values(ascending=False)

plt.figure(figsize=(12, 6))
pca_importance.plot(kind='bar', color='red')
plt.title('Feature Importance Based on First PCA Component')
plt.xlabel('Features')
plt.ylabel('PCA Loading (absolute value)')
plt.xticks(rotation=90)
plt.tight_layout()
plt.savefig("plots/pca_feature_importance.png", dpi=300)
plt.close()

# Histogram for Feature Distributions
df.hist(figsize=(15, 12), bins=50, edgecolor="black")
plt.suptitle("Feature Distribution Histograms")
plt.savefig("plots/feature_histograms.png", dpi=300)
plt.close()

print("Feature importance visualization complete. Plots saved in 'plots/' directory.")


Total features: 45
Feature importance visualization complete. Plots saved in 'plots/' directory.


In [3]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import mutual_info_classif
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Ensure output directories exist
os.makedirs('plots', exist_ok=True)
os.makedirs('text_outputs', exist_ok=True)

# Reload dataset
df = pd.read_csv("processed_training_dataset.csv")

# Check feature names
feature_names = df.columns.tolist()
with open("text_outputs/feature_list.txt", "w") as f:
    f.write(f"Total features: {len(feature_names)}\n")
    f.write("\n".join(feature_names))

# Compute feature correlation heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(df.corr(), cmap='coolwarm', annot=False, fmt=".2f")
plt.title("Feature Correlation Heatmap")
plt.savefig("plots/feature_correlation_heatmap.png", dpi=300)
plt.close()

# Save correlation matrix in text format
correlation_matrix = df.corr()
correlation_matrix.to_csv("text_outputs/correlation_matrix.txt", sep="\t")

# Compute Mutual Information scores (if Attack_label is available)
if 'Attack_label' in df.columns:
    X = df.drop('Attack_label', axis=1)
    y = df['Attack_label']
    
    mi_scores = mutual_info_classif(X, y, random_state=42)
    mi_scores = pd.Series(mi_scores, index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)

    # Save MI scores in text format
    mi_scores.to_csv("text_outputs/mutual_information_scores.txt", sep="\t", header=["Mutual Information Score"])

    # Plot MI scores
    plt.figure(figsize=(12, 6))
    mi_scores.plot(kind='bar', color='blue')
    plt.title('Feature Importance Based on Mutual Information')
    plt.xlabel('Features')
    plt.ylabel('Mutual Information Score')
    plt.xticks(rotation=90)
    plt.tight_layout()
    plt.savefig("plots/mutual_information_distribution.png", dpi=300)
    plt.close()
else:
    print("Warning: 'Attack_label' not found, skipping MI computation.")

# PCA Analysis
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df)

pca = PCA()
pca.fit(df_scaled)

# Scree Plot (Explained Variance)
plt.figure(figsize=(12, 6))
plt.plot(range(1, len(pca.explained_variance_ratio_) + 1), 
         pca.explained_variance_ratio_, 'o-', linewidth=2, color='blue')
plt.title('Scree Plot')
plt.xlabel('Principal Component')
plt.ylabel('Variance Explained')
plt.grid()
plt.savefig("plots/pca_scree_plot.png", dpi=300)
plt.close()

# Cumulative Explained Variance Plot
cumulative_variance = np.cumsum(pca.explained_variance_ratio_)
plt.figure(figsize=(12, 6))
plt.plot(range(1, len(cumulative_variance) + 1), cumulative_variance, 'o-', linewidth=2, color='blue')
plt.axhline(y=0.95, color='r', linestyle='--', label='95% Explained Variance')
plt.title('Cumulative Explained Variance')
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Variance')
plt.legend()
plt.grid()
plt.savefig("plots/pca_cumulative_variance.png", dpi=300)
plt.close()

# Save PCA variance in text format
with open("text_outputs/pca_explained_variance.txt", "w") as f:
    f.write("Principal Component\tExplained Variance Ratio\n")
    for i, variance in enumerate(pca.explained_variance_ratio_):
        f.write(f"{i+1}\t{variance:.6f}\n")

# PCA Feature Importance
pca_importance = np.abs(pca.components_[0])
pca_importance = pd.Series(pca_importance, index=feature_names)
pca_importance = pca_importance.sort_values(ascending=False)

# Save PCA feature importance in text format
pca_importance.to_csv("text_outputs/pca_feature_importance.txt", sep="\t", header=["PCA Loading (absolute value)"])

# Plot PCA-based feature importance
plt.figure(figsize=(12, 6))
pca_importance.plot(kind='bar', color='red')
plt.title('Feature Importance Based on First PCA Component')
plt.xlabel('Features')
plt.ylabel('PCA Loading (absolute value)')
plt.xticks(rotation=90)
plt.tight_layout()
plt.savefig("plots/pca_feature_importance.png", dpi=300)
plt.close()

# Histogram for Feature Distributions
df.hist(figsize=(15, 12), bins=50, edgecolor="black")
plt.suptitle("Feature Distribution Histograms")
plt.savefig("plots/feature_histograms.png", dpi=300)
plt.close()

print("Feature importance visualization complete. Plots saved in 'plots/' directory.")
print("Text data saved in 'text_outputs/' directory.")


Feature importance visualization complete. Plots saved in 'plots/' directory.
Text data saved in 'text_outputs/' directory.


### **Feature Selection for Layer 1 Training**  

#### **Overview**  
To optimize the **Autoencoder-VAE** model for anomaly detection, we performed **feature selection** using:  
1. **Mutual Information (MI)** → Measures dependency between each feature and anomalies.  
2. **Principal Component Analysis (PCA)** → Identifies features contributing to variance.  
3. **Combined Scoring (70% MI + 30% PCA)** → Ensures a balanced feature selection approach.  

---

#### **Selected Features**  
Based on the combined MI-PCA scores, the following **13 features** have been selected:  

| Feature Name               | Description |
|----------------------------|-------------|
| `tcp.dstport_category`     | Categorized TCP destination port range |
| `mbtcp.trans_id`           | Modbus transaction ID (potential protocol-based anomaly) |
| `tcp.ack`                  | TCP acknowledgment flag |
| `mqtt.ver`                 | MQTT protocol version |
| `tcp.connection.synack`     | SYN-ACK flag in TCP handshake |
| `mbtcp.len`                | Length of Modbus TCP payload |
| `mqtt.conflags`            | MQTT connection flags |
| `mqtt.conack.flags`        | MQTT connection acknowledgment flags |
| `tcp.connection.rst`       | TCP Reset flag (can indicate network attacks) |
| `http.tls_port`            | Port used for TLS traffic |
| `tcp.srcport`              | TCP source port |
| `tcp.connection.fin`       | TCP FIN flag (connection termination) |
| `mqtt.hdrflags`            | MQTT header flags |

---

#### **Next Steps**  
**Dataset Preparation**  
- Filter `processed_training_dataset.csv` to retain only these **13 features**.  
- Normalize values using **MinMaxScaler**.  
- Save as **`layer1_training_data.csv`** for training.  

**Autoencoder-VAE Model Setup**  
- **Input Size:** 13  
- **Latent Space:** 6 (50% of input)  
- **Loss Function:** Reconstruction Loss + KL Divergence  
- **Activation:** ReLU for hidden layers, Sigmoid for output  

**Training Phase**  
- Train using **only normal data** to model normal patterns.  
- Implement **Early Stopping & Learning Rate Decay** for stability.  
- Save the trained **encoder model** for feature extraction in Layer 2.  

---


Data_preparation for Layer one

In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import pickle
import os

def prepare_layer1_data():
    print("Starting Layer 1 data preparation...")
    
    # Create output directory if it doesn't exist
    output_dir = "layer1_output"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        print(f"Created output directory: {output_dir}")
    else:
        print(f"Output directory already exists: {output_dir}")
    
    # 1. Load the dataset
    print("Loading processed_training_dataset.csv...")
    try:
        df = pd.read_csv('processed_training_dataset.csv')
        print(f"Original dataset loaded: {df.shape[0]} rows and {df.shape[1]} columns")
    except FileNotFoundError:
        print("Error: processed_training_dataset.csv not found.")
        return
    
    # 2. Filter to keep only the 13 selected features
    selected_features = [
        'tcp.dstport_category', 'mbtcp.trans_id', 'tcp.ack', 'mqtt.ver',
        'tcp.connection.synack', 'mbtcp.len', 'mqtt.conflags', 'mqtt.conack.flags',
        'tcp.connection.rst', 'http.tls_port', 'tcp.srcport', 'tcp.connection.fin',
        'mqtt.hdrflags'
    ]
    
    # Check if all selected features exist in the dataset
    missing_features = [feature for feature in selected_features if feature not in df.columns]
    if missing_features:
        print(f"Warning: The following features are missing from the dataset: {missing_features}")
        # Continue with available features
        selected_features = [feature for feature in selected_features if feature in df.columns]
        
    # Filter to keep only selected features
    df_filtered = df[selected_features]
    print(f"Filtered dataset: {df_filtered.shape[0]} rows and {df_filtered.shape[1]} columns")
    
    # 3. Handle missing values with median
    print("Handling missing values...")
    missing_before = df_filtered.isnull().sum().sum()
    
    if missing_before > 0:
        # Replace missing values with median
        for column in df_filtered.columns:
            if df_filtered[column].isnull().sum() > 0:
                median_value = df_filtered[column].median()
                df_filtered[column].fillna(median_value, inplace=True)
                
    missing_after = df_filtered.isnull().sum().sum()
    print(f"Missing values: {missing_before} before, {missing_after} after replacement")
    
    # 4. Apply MinMaxScaler
    print("Applying MinMaxScaler...")
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(df_filtered)
    
    # Convert scaled data back to DataFrame
    df_scaled = pd.DataFrame(scaled_data, columns=df_filtered.columns)
    
    # 5. Save the processed dataset to the output directory
    output_csv_path = os.path.join(output_dir, 'layer1_training_data.csv')
    print(f"Saving processed dataset to {output_csv_path}...")
    df_scaled.to_csv(output_csv_path, index=False)
    
    # 6. Save the scaler model to the output directory
    output_scaler_path = os.path.join(output_dir, 'scaler_layer1.pkl')
    print(f"Saving scaler model to {output_scaler_path}...")
    with open(output_scaler_path, 'wb') as file:
        pickle.dump(scaler, file)
    
    # Save feature names list for reference
    output_features_path = os.path.join(output_dir, 'layer1_features.txt')
    with open(output_features_path, 'w') as file:
        for feature in df_filtered.columns:
            file.write(f"{feature}\n")
    
    # 7. Print dataset summary
    print("\n========== Layer 1 Dataset Summary ==========")
    print(f"Total rows: {df_scaled.shape[0]}")
    print(f"Total columns: {df_scaled.shape[1]}")
    print("\nFeature statistics after preprocessing:")
    print(df_scaled.describe())
    print("\nFirst 5 rows of the processed dataset:")
    print(df_scaled.head())
    
    # Save summary statistics to a report file
    summary_path = os.path.join(output_dir, 'layer1_summary_report.txt')
    with open(summary_path, 'w') as file:
        file.write("========== Layer 1 Dataset Summary ==========\n")
        file.write(f"Total rows: {df_scaled.shape[0]}\n")
        file.write(f"Total columns: {df_scaled.shape[1]}\n\n")
        file.write("Feature statistics after preprocessing:\n")
        file.write(df_scaled.describe().to_string())
        
    print("\nLayer 1 data preparation completed successfully.")
    print(f"Output files created in the '{output_dir}' directory:")
    print(f"  - {os.path.basename(output_csv_path)}")
    print(f"  - {os.path.basename(output_scaler_path)}")
    print(f"  - {os.path.basename(output_features_path)}")
    print(f"  - {os.path.basename(summary_path)}")

if __name__ == "__main__":
    prepare_layer1_data()

Starting Layer 1 data preparation...
Created output directory: layer1_output
Loading processed_training_dataset.csv...
Original dataset loaded: 150000 rows and 45 columns
Filtered dataset: 150000 rows and 13 columns
Handling missing values...
Missing values: 0 before, 0 after replacement
Applying MinMaxScaler...
Saving processed dataset to layer1_output/layer1_training_data.csv...
Saving scaler model to layer1_output/scaler_layer1.pkl...

========== Layer 1 Dataset Summary ==========
Total rows: 150000
Total columns: 13

Feature statistics after preprocessing:
       tcp.dstport_category  mbtcp.trans_id       tcp.ack       mqtt.ver  \
count          150000.00000   150000.000000  1.500000e+05  150000.000000   
mean                0.71954        0.009242  5.768431e-02       0.023978   
std                 0.25203        0.037664  1.956340e-01       0.110854   
min                 0.00000        0.000000  0.000000e+00       0.000000   
25%                 0.50000        0.000000  2.463365